## Read Text Data From File
<ul>
<li> remove label 'handle', 'tweet'</li>
    <li> encoding = 'utf-8'</li>
    <li> labelList:  Clinton: 1 ; Trump: 0</li>
    <li> print size</li>
</ul>

In [2]:
## read text data from file

import csv
import string

labelList = []
tweetList = []
tweetList_test = []

with open('train.csv', encoding='utf-8') as trainfile:
    reader = csv.reader(trainfile)
    for row in reader:
        if row[0] == 'HillaryClinton':
            labelList.append(1)
        elif row[0] == 'realDonaldTrump':
            labelList.append(0)
            
        if row[1] != 'tweet':
            tweetList.append(row[1])
            
with open('test.csv', encoding='utf-8') as testfile:
    reader = csv.reader(testfile)
    for row in reader: 
        if row[1] != 'tweet':
            tweetList_test.append(row[1])

N = len(tweetList)
print(len(tweetList))
print(len(labelList))

N_test = len(tweetList_test)
print(len(tweetList_test))

4743
4743
1701


## Create Corpus
<ul>
    <li>unique word in corpus</li>
</ul>

In [3]:
## corpus
import string

def refineWordsList(tweetList):
    ret = []
    for tweet in tweetList:
        step1 = tweet.split('http',1)[0]
        step2 = step1.lower()
        step3 = step2.translate(str.maketrans('.',' '))
        step4 = step3.translate(str.maketrans('','',string.punctuation))
        ret.append(step4)
    return ret

tweetList = refineWordsList(tweetList)
tweetList_test = refineWordsList(tweetList_test)

corpus = set()

for tweet in tweetList:
    words = tweet.split();
    corpus.update(words)

corpus = list(corpus)
M = len(corpus)
print(M)

8787


## Label Encoding

<ul>
    <li>hash table encoder each word to unique number</li>
    <li>encode entire tweet list</li>
    <li>decode method</li>
</ul>

In [4]:
## Label Encoding

hashEncoder = {}
i = 0
for word in corpus:
    hashEncoder[word] = i
    i += 1

# train
tweetList_encoded = []
for tweet in tweetList:
    tweet_encoded = []
    for word in tweet.split():
        word_encoded = hashEncoder[word]
        tweet_encoded.append(word_encoded)
    tweetList_encoded.append(tweet_encoded)
    
# test
tweetList_test_encoded = []
for tweet in tweetList_test:
    tweet_encoded = []
    for word in tweet.split():
        if hashEncoder.get(word) != None :
            word_encoded = hashEncoder[word]
            tweet_encoded.append(word_encoded)
    tweetList_test_encoded.append(tweet_encoded)

def decodeTweet(tweet_encoded):
    decodeList = []
    for word_encoded in tweetList_encoded[5]:
        decodeList.append(list(hashEncoder.keys())[list(hashEncoder.values()).index(word_encoded)])
    return decodeList
    
print(len(tweetList_encoded))
print(len(tweetList_test_encoded))
print()
print(tweetList[5])
print(tweetList_encoded[5])
print(decodeTweet(tweetList_encoded[5]))
print()
print(tweetList_test[5])
print(tweetList_test_encoded[5])

4743
1701

when donald trump goes low   register to vote 
[3207, 1223, 5524, 2760, 7080, 7493, 4844, 6346]
['when', 'donald', 'trump', 'goes', 'low', 'register', 'to', 'vote']

its nationalvoterregistrationday  celebrate by registering to vote → 
[3957, 1464, 4302, 5589, 2686, 4844, 6346, 7892]


In [14]:
tweetList_encoded_train = [tweetList_encoded[x] for x in range ((int)(0.8 * N))]
tweetList_encoded_valid = [tweetList_encoded[x] for x in range ((int)(0.8 * N) + 1, N)]

## import

In [3]:
import tensorflow as tf
import numpy as np
import time
import math
from tensorflow.contrib import rnn

## LSTM

In [1]:
featureNum = 20
batch_size = 128

In [ ]:
x = tf.placeholder(tf.int32, [batch_size, None])

In [7]:
embeddingMatrix = tf.Variable(tf.random_normal([M, featureNum], stddev=0.35), name = 'embedding')
inputMatrix = tf.nn.embedding_lookup(embeddingMatrix, x)

In [ ]:
lstm_cell = rnn.BasicLSTMCell(lengthOfSentence)
outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

## FeedForwardNN

In [ ]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 10 
num_input = 
num_classes = 2

In [ ]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=, 
    y=,
    batch_size=batch_size, 
    num_epochs=None, 
    shuffle=True)

In [ ]:
# Define the neural network
def neural_net(x):
    layer_1 = tf.layers.dense(x, n_hidden_1)
    out_layer = tf.layers.dense(layer_1, num_classes)
    return out_layer

In [ ]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs